In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")


gams_python_path="/Library/Frameworks/GAMS.framework/Resources/apifiles/Python/gams"

In [2]:
model=pd.read_csv('/Users/hannahkamen/Downloads/windc_ind.csv')

In [6]:
# model[model['Row_Name']=='Other real estate']

In [7]:
model[model['gams.IOCode'].isin(['ppd', 'res', 'com', 'rec', 'con', 'agr', 'eec', 'pub', 'ins',
       'tex', 'leg', 'fen', 'uti', 'nmp', 'brd', 'bnk', 'ore', 'man',
       'mch', 'dat', 'amd', 'rnt', 'pla', 'tsv', 'sec', 'art', 'mov',
       'fpd', 'pri', 'grd', 'sle', 'osv', 'fmt', 'pet', 'cep', 'wst',
       'adm', 'pmt', 'wpd', 'wrh', 'che', 'air', 'mmf'])]['Row_Name'].unique()

array(['Farms', 'Wood products',
       'Amusements, gambling, and recreation industries', 'Construction',
       'Computer systems design and related services',
       'Petroleum and coal products',
       'Miscellaneous professional, scientific, and technical services',
       'Nonmetallic mineral products', 'Chemical products',
       'Fabricated metal products', 'Utilities',
       'State and local government enterprises',
       'Furniture and related products', 'Miscellaneous manufacturing',
       'Textile mills and textile product mills', 'Paper products',
       'Plastics and rubber products', 'Machinery',
       'Computer and electronic products',
       'Electrical equipment, appliances, and components',
       'Primary metals', 'Printing and related support activities',
       'Other services, except government',
       'Administrative and support services', 'Warehousing and storage',
       'Rental and leasing services and lessors of intangible assets',
       'Federal Res

#### Import Data

In [2]:
model=pd.read_csv('/Users/hannahkamen/Downloads/windc_ind.csv')
#cps=pd.read_stata('/Users/hannahkamen/Downloads/acs_20132018_5yr.dta')
cps=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/acs5yr_0610.dta')
lookup=pd.read_excel('/Users/hannahkamen/Downloads/windc_cps_industry_lookup.xlsx')
lookup_final=pd.read_excel('/Users/hannahkamen/Downloads/dropped_merge.xlsx')
state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/state_lookup.xlsx')

lookup['NAICS']=lookup['NAICS'].astype(str).str.strip()
lookup_final['NAICS']=lookup_final['NAICS'].astype(str).str.strip()
lookup_final['cps_code']=lookup_final['cps_code'].astype(str).str.strip()

#cps = np.loadtxt('/Users/hannahkamen/Downloads/usa_00002.dat', unpack = True)



In [5]:
state_lookup['state']=state_lookup['state'].str.strip()
state_lookup['abbrev']=state_lookup['abbrev'].str.strip()

In [3]:
###read in dropped 

In [4]:
cps_codes=pd.DataFrame(data={'cps_code':cps['indnaics'].str.strip().unique()})
windc_codes=pd.DataFrame(data={'windc_code':model['IOCode'].str.strip().unique()})

In [5]:
####match on first three symbols
cps_codes['cps_code3']=cps_codes['cps_code'].str[0:3]
lookup['NAICS3']=lookup['NAICS'].str[0:3]

merge3=lookup.merge(cps_codes, left_on='NAICS3',right_on='cps_code3',how='inner',indicator=True)

In [6]:
###append manual lookup to fill gaps
merge_f=merge3[['NAICS','cps_code']].append(lookup_final)

In [7]:
###get state symbol for place of residence
cps['statefip']=cps['statefip'].str.lower().str.strip()
cps['pwstate2']=cps['pwstate2'].str.lower().str.strip()

cps_m=cps.merge(merge_f, left_on='indnaics', right_on='cps_code',how='left').merge(state_lookup, left_on='statefip',right_on='state', how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_residence'})

###get state symbol for place of work
cps_m=cps_m.merge(state_lookup, left_on='pwstate2', right_on='state',how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_work'})

### get skilled unskilled breakouts
cps_m['educ_new']=np.where(cps_m['educ'].isin(['2 years of college','4 years of college','5+ years of college']),'skl','unskl')
 


In [8]:
###prep household variable
cps_m['hh']=np.where(cps_m['inctot']<=25000,'hh1','')
cps_m['hh']=np.where(((cps_m['inctot']>25000) & (cps_m['inctot']<50000)),'hh2',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=50000) & (cps_m['inctot']<75000)),'hh3',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=75000) & (cps_m['inctot']<150000)),'hh4',cps_m['hh'])
cps_m['hh']=np.where((cps_m['inctot']>=150000),'hh5',cps_m['hh'])

In [9]:
non_s=['Output',
       'imports', 'ciffob', 'BasicSupply', 'Margins', 'TrnCost', 'TrdTrn',
       'Duties', 'Tax', 'Subsidies', 'TaxLesSubsidies', 'Supply']

states=cps_m['state_residence'].unique()

In [10]:
####limit to workers with location assigned for work and residence
cps_m1=cps_m[(cps_m['state_work'].isin(states))& (cps_m['state_residence'].isin(states))& (cps_m['age']!='90 (90+ in 1980 and 1990)')]


In [86]:
cps_m1['age']=cps_m1['age'].astype(int)

cps_m1['age_id']=np.where(((cps_m1['age']>=25) & (cps_m1['age']<30)),'2','0')

cps_m1['age_id']=np.where(((cps_m1['age']>=30) & (cps_m1['age']<35)),'3',cps_m1['age_id'])

cps_m1['age_id']=np.where(((cps_m1['age']>=35) & (cps_m1['age']<40)),'4',cps_m1['age_id'])

cps_m1['age_id']=np.where(((cps_m1['age']>=40) & (cps_m1['age']<45)),'5',cps_m1['age_id'])

cps_m1['age_id']=np.where(((cps_m1['age']>=45) & (cps_m1['age']<50)),'6',cps_m1['age_id'])

cps_m1['age_id']=np.where(((cps_m1['age']>=50) & (cps_m1['age']<55)),'7',cps_m1['age_id'])

In [92]:
#####get educ and age splits by state
cps_state_split=cps_m1.groupby(['educ_new','age_id','statefip','state_residence'],as_index=False).agg({'incwage':'count'})

##total
cps_state_split_tot=cps_m1.groupby(['statefip'],as_index=False).agg({'incwage':'count'})

cps_state_split=cps_state_split.merge(cps_state_split_tot,on='statefip')

cps_state_split['state_prop']=cps_state_split['incwage_x']/cps_state_split['incwage_y']

state_split=cps_state_split.pivot(index='statefip', columns=['educ_new','age_id'], values='state_prop').reset_index()
state_split.columns=["_".join((j,k)) for (j,k) in state_split.columns]

state_pop=pd.read_excel('/Users/hannahkamen/Downloads/state_pop.xlsx')
state_pop['state']=state_pop['state'].str.strip()
state_split0=state_split.merge(state_pop,left_on='statefip_',right_on='state')
state_split0.to_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')

In [94]:
#####get age splits by state
cps_state_split=cps_m1.groupby(['age_id','statefip','state_residence'],as_index=False).agg({'incwage':'count'})

##total
cps_state_split_tot=cps_m1.groupby(['statefip'],as_index=False).agg({'incwage':'count'})

cps_state_split=cps_state_split.merge(cps_state_split_tot,on='statefip')

cps_state_split['state_prop']=cps_state_split['incwage_x']/cps_state_split['incwage_y']

state_split=cps_state_split.pivot(index='statefip', columns=['age_id'], values='state_prop').reset_index()
#state_split.columns=["_".join((j,k)) for (j,k) in state_split.columns]

state_pop=pd.read_excel('/Users/hannahkamen/Downloads/state_pop.xlsx')
state_pop['state']=state_pop['state'].str.strip()
state_split0=state_split.merge(state_pop,left_on='statefip',right_on='state')
state_split0.to_excel('/Users/hannahkamen/Downloads/state_age_shares.xlsx')

In [11]:
#####get educ splits by state
cps_state_split=cps_m1.groupby(['educ_new','statefip','state_residence'],as_index=False).agg({'incwage':'count'})

##total
cps_state_split_tot=cps_m1.groupby(['statefip'],as_index=False).agg({'incwage':'count'})

cps_state_split=cps_state_split.merge(cps_state_split_tot,on='statefip')

cps_state_split['state_prop']=cps_state_split['incwage_x']/cps_state_split['incwage_y']

state_split=cps_state_split.pivot(index='statefip', columns=['educ_new'], values='state_prop').reset_index()
#state_split.columns=["_".join((j,k)) for (j,k) in state_split.columns]

state_pop=pd.read_excel('/Users/hannahkamen/Downloads/state_pop.xlsx')
state_pop['state']=state_pop['state'].str.strip()
state_split0=state_split.merge(state_pop,left_on='statefip',right_on='state')
state_split0.to_excel('/Users/hannahkamen/Downloads/state_educ_shares.xlsx')

In [12]:
state_split0

,statefip,skl,unskl,state,state_pop
0,alabama,0.394368,0.605632,alabama,4779736
1,alaska,0.392314,0.607686,alaska,710231
2,arizona,0.433231,0.566769,arizona,6392017
3,arkansas,0.318447,0.681553,arkansas,2915918
4,california,0.485955,0.514045,california,37253956
5,colorado,0.524221,0.475779,colorado,5029196
6,connecticut,0.530862,0.469138,connecticut,3574097
7,delaware,0.444458,0.555542,delaware,897934
8,district of columbia,0.725673,0.274327,district of columbia,601723
9,florida,0.447951,0.552049,florida,18801310


In [74]:
cps_state_split.pivot?

In [91]:
state_split.head()

,statefip_,skl_0,skl_2,skl_3,skl_4,skl_5,skl_6,skl_7,unskl_0,unskl_2,unskl_3,unskl_4,unskl_5,unskl_6,unskl_7
0,alabama,0.105404,0.039066,0.043793,0.049895,0.050821,0.052949,0.052439,0.222783,0.049659,0.051701,0.061713,0.068419,0.077842,0.073515
1,alaska,0.111969,0.033725,0.037637,0.041896,0.050265,0.054326,0.062497,0.211063,0.052394,0.054326,0.057149,0.067102,0.085772,0.079879
2,arizona,0.120677,0.043527,0.049146,0.053954,0.052812,0.055992,0.057122,0.205752,0.055556,0.054534,0.058304,0.061684,0.066981,0.063957
3,arkansas,0.085387,0.034571,0.038225,0.039881,0.039840,0.039991,0.040552,0.256080,0.057044,0.058987,0.064886,0.074959,0.084375,0.085223
4,california,0.131817,0.049547,0.055820,0.060629,0.060760,0.064456,0.062925,0.178081,0.049052,0.048254,0.054431,0.060285,0.064166,0.059776


### Assign Sector Values and Limit Data

In [ ]:
#list of US states

#### drop if unemployed
#cps_m1=cps_m[(cps_m['indnaics']!='0') & (cps_m['indnaics']!='') & (cps_m['state_work'].isin(states))]

#### group data by all relevant breakous - r,q,h,s,sk
all_breakouts0=cps_m1.reset_index()[['state_residence','state_work','NAICS','hh','educ_new','incwage']].groupby(['state_residence','state_work','NAICS','hh','educ_new'],as_index=False).agg({'incwage':sum})

#rename variables and limit dataframe to relevant sectors
all_breakouts0=all_breakouts0.rename(columns={'state_residence':'r','state_work':'q','NAICS':'s','hh':'h','educ_new':'sk'})
all_breakouts0=all_breakouts0[['r','q','s','h','sk','incwage']]

### now group model dataset by relevant Naics codes and Windc specific codes
model_gr=model.groupby(['IOCode','gams.IOCode'],as_index=False).sum()
model_gr=model_gr[['IOCode','gams.IOCode']]

###merge with cps data 
all_breakouts0=all_breakouts0.merge(model_gr, left_on='s',right_on='IOCode',how='left')

##get rid of used sector

all_breakouts0=all_breakouts0[all_breakouts0['gams.IOCode']!='use']

##now delete old NAICS var and re-assign ind var as the values in 'gams.IOCode'
del all_breakouts0['s']
del all_breakouts0['IOCode']
all_breakouts0=all_breakouts0.rename(columns={'gams.IOCode':'s'})
all_breakouts=all_breakouts0[['r','q','s','h','sk','incwage']]



In [ ]:
all_breakouts.head()

#### get current model breakouts region lived in,region worked, household type le0(r,q,h)

In [ ]:
###classify census regions
midwest=['IA','OH','WI','NE','IL','MI','SD','ND','MN','MO','IN','KS']
south=['FL','MD','TN','WV','OK','KY','NC','VA','DE','GA','MS','TX','AL','LA','AR','SC','DC']
west=['AK','AZ','NM','HI','CA','WA','NV','OR','ID','UT','MT','WY','CO']
northeast=['VT','NH','CT','ME','MA','NY','NJ','PA','RI']

###get census region lived in, census region worked in

all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(midwest),'midwest','')
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(south),'south',all_breakouts['census_region_r'])
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(west),'west',all_breakouts['census_region_r'])
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(northeast),'northeast',all_breakouts['census_region_r'])


all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(midwest),'midwest','')
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(south),'south',all_breakouts['census_region_q'])
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(west),'west',all_breakouts['census_region_q'])
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(northeast),'northeast',all_breakouts['census_region_q'])

####limit to where person works in same census region that they live in
all_breakouts_lm=all_breakouts[(all_breakouts['census_region_r']==all_breakouts['census_region_q'])]

###get total income by r,q then get income by r,q,h calculate to get share household income by income type
#total by rqh
all_breakouts_lm_gr=all_breakouts_lm.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})

##total by r,q
# all_breakouts_lm_tot=all_breakouts_lm.groupby(['r','q'],as_index=False).agg({'incwage':sum})

# all_breakouts_lm_tot=all_breakouts_lm_tot.rename(columns={'incwage':'incwage_total'})

# le0_breakouts=all_breakouts_lm_gr.merge(all_breakouts_lm_tot, on=['r','q'],how='left')

# le0_breakouts['hh_shr']=le0_breakouts['incwage']/le0_breakouts['incwage_total']
le0_breakouts=all_breakouts_lm.groupby(['census_region_r','h'],as_index=False).agg({'incwage':sum})


###get total income by r,s then get income by r calculate to get share of labor expenditures in each regionfor each sector

all_breakouts_lm_gr1=all_breakouts_lm.groupby(['r','s'],as_index=False).agg({'incwage':sum})

##total by r,q
# all_breakouts_lm_tot1=all_breakouts_lm.groupby(['r'],as_index=False).agg({'incwage':sum})

# all_breakouts_lm_tot1=all_breakouts_lm_tot1.rename(columns={'incwage':'incwage_total'})

# ld0_breakouts=all_breakouts_lm_gr1.merge(all_breakouts_lm_tot1, on=['r'],how='left')

# ld0_breakouts['r_shr']=ld0_breakouts['incwage']/ld0_breakouts['incwage_total']
ld0_breakouts=all_breakouts_lm.groupby(['census_region_q','s'],as_index=False).agg({'incwage':sum})



#### Get missing combinations for both datasets

In [ ]:
####gams labor demand data
gams_dta_ld0=pd.read_csv('/Users/hannahkamen/Downloads/windc_ld0_d2.csv')

####gams labor endowment data
gams_dta_le0=pd.read_csv('/Users/hannahkamen/Downloads/windc_le0_d.csv')

In [ ]:
###group labor endowment by place of work and sector. These are all the place of work and sector combinations
### in the model that we need shares for

gams_dta_ld0=gams_dta_ld0.rename(columns={' flx':'q',' int':'s',' scn':'sk',' esubw_oth':'pop'})

gams_dta_ld0['census_region_q']=np.where(gams_dta_ld0['q'].isin(midwest),'midwest','')
gams_dta_ld0['census_region_q']=np.where(gams_dta_ld0['q'].isin(south),'south',gams_dta_ld0['census_region_q'])
gams_dta_ld0['census_region_q']=np.where(gams_dta_ld0['q'].isin(west),'west',gams_dta_ld0['census_region_q'])
gams_dta_ld0['census_region_q']=np.where(gams_dta_ld0['q'].isin(northeast),'northeast',gams_dta_ld0['census_region_q'])

ld00=gams_dta_ld0[['q','s','sk','pop','census_region_q']]
ld0=gams_dta_ld0.rename(columns={' flx':'q',' int':'s',' esubw_oth':'pop'})[['census_region_q','s','pop']].groupby(['census_region_q','s'],as_index=False).sum()

##prepare le0 table

gams_dta_le0=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' esubw_oth':'sk',' sector':'pop'})

gams_dta_le0['census_region_r']=np.where(gams_dta_le0['r'].isin(midwest),'midwest','')
gams_dta_le0['census_region_r']=np.where(gams_dta_le0['r'].isin(south),'south',gams_dta_le0['census_region_r'])
gams_dta_le0['census_region_r']=np.where(gams_dta_le0['r'].isin(west),'west',gams_dta_le0['census_region_r'])
gams_dta_le0['census_region_r']=np.where(gams_dta_le0['r'].isin(northeast),'northeast',gams_dta_le0['census_region_r'])

gams_dta_le0['census_region_q']=np.where(gams_dta_le0['q'].isin(midwest),'midwest','')
gams_dta_le0['census_region_q']=np.where(gams_dta_le0['q'].isin(south),'south',gams_dta_le0['census_region_r'])
gams_dta_le0['census_region_q']=np.where(gams_dta_le0['q'].isin(west),'west',gams_dta_le0['census_region_r'])
gams_dta_le0['census_region_q']=np.where(gams_dta_le0['q'].isin(northeast),'northeast',gams_dta_le0['census_region_r'])


le00=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' sector':'pop'})[['r','q','sk','census_region_r','h','pop','census_region_q']]
le0=le00.groupby(['census_region_r','h'],as_index=False).sum()
## merge region lived in and region worked in combinations with all sector combinations for the region worked in (q)

le0_matches=le0.merge(le0_breakouts,on=['census_region_r','h'],how='outer',indicator=True)
le0_matches=le0_matches[le0_matches['_merge']!='both']
le0_matches=le0_matches[['census_region_r','h','pop','incwage','_merge']]

ld0_matches=ld0.merge(ld0_breakouts,on=['census_region_q','s'],how='outer',indicator=True)
ld0_matches=ld0_matches[ld0_matches['_merge']!='both']
ld0_matches=ld0_matches[['census_region_q','s','pop','incwage','_merge']]




#### Scale ACS data to make each household the same weight as in Windc and each sector the same weight as Windc in each region

#### Get Deflator by r,q,h,s 

In [ ]:
#######g

####get proportion household in their working region in GAMS data

le00_rh=le00.groupby(['census_region_q','r','q','h'],as_index=False).sum()
le00_rh=le00_rh.rename(columns={'pop':'pop_gams'})

####get proportion household in their working region in ACS data

acs_cons=all_breakouts_lm.groupby(['census_region_q','r','q','h'],as_index=False).sum()
acs_cons=acs_cons.rename(columns={'incwage':'pop'})

####get decompressor

total_deflator_by_region=acs_cons.merge(le00_rh,on=['census_region_q','r','q','h'])
total_deflator_by_region['deflator']=total_deflator_by_region['pop_gams']/total_deflator_by_region['pop']
cons_df=total_deflator_by_region[['r','q','census_region_q','h','deflator']]


#### Get Deflator by q,s

In [ ]:
#######g

####get proportion household in their working region in GAMS data

ld00_qs=ld00.groupby(['census_region_q','q','s'],as_index=False).sum()
ld00_qs=ld00_qs.rename(columns={'pop':'pop_gams'})

####get proportion household in their working region in ACS data

acs_prod=all_breakouts_lm.groupby(['census_region_q','q','s'],as_index=False).sum()
acs_prod=acs_prod.rename(columns={'incwage':'pop'})

####get decompressor

total_deflator_by_region=acs_prod.merge(ld00_qs,on=['census_region_q','q','s'])
total_deflator_by_region['deflator']=total_deflator_by_region['pop_gams']/total_deflator_by_region['pop']
prod_df=total_deflator_by_region[['q','census_region_q','s','deflator']]


#### Deflate endowment and demand by skill type

In [ ]:
####CONS
###breakout wages by skill type
acs_cons_sk=all_breakouts_lm.groupby(['r','census_region_q','q','h','sk'],as_index=False).agg({'incwage':sum})
###merge deflator on
acs_cons_sk=acs_cons_sk.merge(cons_df,on=['r','census_region_q','q','h'])
acs_cons_sk['pop_adjust']=acs_cons_sk['incwage']*acs_cons_sk['deflator']
acs_cons_sk=acs_cons_sk[['r','census_region_q','q','h','sk','pop_adjust']]

###get total spend by household type and regions
acs_cons_tot=all_breakouts_lm.groupby(['r','census_region_q','q','h'],as_index=False).agg({'incwage':sum})
acs_cons_tot=acs_cons_tot.rename(columns={'incwage':'pop_total'})

###merge deflator on
acs_cons_tot=acs_cons_tot.merge(cons_df,on=['r','census_region_q','q','h'])
acs_cons_tot['pop_total_adjust']=acs_cons_tot['pop_total']*acs_cons_tot['deflator']
acs_cons_tot=acs_cons_tot[['r','census_region_q','q','h','pop_total_adjust']]

####merge adjusted spend by skill onto adjusted total spend by household
acs_cons_adj=acs_cons_sk.merge(acs_cons_tot,on=['r','census_region_q','q','h'])
acs_cons_adj['expenditure_share']=acs_cons_adj['pop_adjust']/acs_cons_adj['pop_total_adjust']

In [ ]:
####PROD
###breakout wages by skill type
acs_prod_sk=all_breakouts_lm.groupby(['census_region_q','q','s','sk'],as_index=False).agg({'incwage':sum})
###merge deflator on
acs_prod_sk=acs_prod_sk.merge(prod_df,on=['census_region_q','q','s'])
acs_prod_sk['pop_adjust']=acs_prod_sk['incwage']*acs_prod_sk['deflator']
acs_prod_sk=acs_prod_sk[['census_region_q','q','s','sk','pop_adjust']]

###get total spend by household type and regions
acs_prod_tot=all_breakouts_lm.groupby(['census_region_q','q','s'],as_index=False).agg({'incwage':sum})
acs_prod_tot=acs_prod_tot.rename(columns={'incwage':'pop_total'})

###merge deflator on
acs_prod_tot=acs_prod_tot.merge(prod_df,on=['census_region_q','q','s'])
acs_prod_tot['pop_total_adjust']=acs_prod_tot['pop_total']*acs_prod_tot['deflator']
acs_prod_tot=acs_prod_tot[['census_region_q','q','s','pop_total_adjust']]

####merge adjusted spend by skill onto adjusted total spend by household
acs_prod_adj=acs_prod_sk.merge(acs_prod_tot,on=['census_region_q','q','s'])
acs_prod_adj['expenditure_share']=acs_prod_adj['pop_adjust']/acs_prod_adj['pop_total_adjust']

In [ ]:
acs_cons_adj.head()

#### Get deflated expenditure shares by Census region Consumer



In [ ]:
##sum by region hh and skill
census_cons=acs_cons_adj.groupby(['census_region_q','h','sk'],as_index=False).agg({'pop_adjust':sum})

#sum by region and hh
census_cons_tot=acs_cons_adj.groupby(['census_region_q','h'],as_index=False).agg({'pop_adjust':sum})
census_cons_tot=census_cons_tot.rename(columns={'pop_adjust':'pop_adjust_total'})

census_cons=census_cons.merge(census_cons_tot, on=['census_region_q','h'])
census_cons['expenditure_share']=census_cons['pop_adjust']/census_cons['pop_adjust_total']
#census_cons=census_cons[['census_region_q','h','sk','expenditure_share']]

#### Get deflated expenditure shares by Census region Producer

In [ ]:
##sum by region hh and skill
census_prod=acs_prod_adj.groupby(['census_region_q','s','sk'],as_index=False).agg({'pop_adjust':sum})

#sum by region and hh
census_prod_tot=acs_prod_adj.groupby(['census_region_q','s'],as_index=False).agg({'pop_adjust':sum})
census_prod_tot=census_prod_tot.rename(columns={'pop_adjust':'pop_adjust_total'})

census_prod=census_prod.merge(census_prod_tot, on=['census_region_q','s'])
census_prod['expenditure_share']=census_prod['pop_adjust']/census_prod['pop_adjust_total']
census_prod=census_prod[['census_region_q','s','sk','expenditure_share']]

##### Merge on all the combinations

In [ ]:
cons_final=le00_rh.merge(census_cons,on=['census_region_q','h'])
cons_final['pop_final']=cons_final['pop_gams']*cons_final['expenditure_share']

In [ ]:
prod_final=ld00_qs.merge(census_prod,on=['census_region_q','s'])
prod_final['pop_final']=prod_final['pop_gams']*prod_final['expenditure_share']

In [ ]:
rules_for_prod.head()

In [ ]:
ld0_testpop.sort_values(by=['q','s']).head()

In [ ]:
prod_final.sort_values(by=['q','s']).head()

In [ ]:
prod_final_gr.head()

In [ ]:
prod_final_adj.sort_values(by=['q','s']).head()

In [ ]:
ld0_testpop.sort_values(by=['q','s']).head()

In [ ]:
##get totals by skill type in each region from producer data
rules_for_prod=cons_final.groupby(['q','census_region_q','sk'],as_index=False).agg({'pop_final':sum})
rules_for_prod=rules_for_prod.rename(columns={'pop_final':'pop_final_cons'})

##group producer data by current demand sums by region
prod_final_gr=prod_final.groupby(['q','census_region_q','sk'],as_index=False).agg({'pop_final':sum})
prod_final_gr=prod_final_gr.rename(columns={'pop_final':'prod_pop_actual'})

##merge
prod_final_adj=prod_final.merge(rules_for_prod,on=['q','census_region_q','sk']).merge(prod_final_gr,on=['q','census_region_q','sk'])

####get by how much the state level labor demand by skill is off from the endowment by skill
prod_final_adj['deflator']=prod_final_adj['pop_final_cons']/prod_final_adj['prod_pop_actual']

prod_final_adj['deflator_pct']=(prod_final_adj['pop_final_cons']-prod_final_adj['prod_pop_actual'])/prod_final_adj['prod_pop_actual']
prod_final_adj['raw_change']=(prod_final_adj['pop_final_cons']-prod_final_adj['prod_pop_actual'])



###adjust 
prod_final_adj['pop_final_adj']=prod_final_adj['pop_final']*prod_final_adj['deflator']

##get sum of new population by sector and region
prod_final_adj_tot=prod_final_adj.groupby(['q','census_region_q','s'],as_index=False).agg({'pop_final_adj':sum})
prod_final_adj_tot=prod_final_adj_tot.rename(columns={'pop_final_adj':'pop_final_adj_total'})


###but pop final adjust total does not equal original gams value for that sector
####get total deflator that sector total is off by 


###calculate new shares
prod_final_adj=prod_final_adj.merge(prod_final_adj_tot,on=['q','census_region_q','s'])
prod_final_adj['expenditure_share_adj']=prod_final_adj['pop_final_adj']/prod_final_adj['pop_final_adj_total']

#####get percentage of total skilled and unskilled that each current value makes up in region

# prod_final_total_pct=prod_final_adj.groupby(['q','sk'],as_index=False).agg({'pop_final':sum})

# prod_final_total_pct=prod_final_total_pct.rename(columns={'pop_final':'pop_final_tot'})

# prod_final_adj=prod_final_adj.merge(prod_final_total_pct,on=['q','sk'])

# prod_final_adj['share_labor_type_demand_old']=prod_final_adj['pop_final']/prod_final_adj['pop_final_tot']


prod_final_total_pct=prod_final_adj.groupby(['q'],as_index=False).agg({'pop_final':sum})

prod_final_total_pct=prod_final_total_pct.rename(columns={'pop_final':'pop_final_tot'})

prod_final_adj=prod_final_adj.merge(prod_final_total_pct,on=['q'])

prod_final_adj['share_labor_in_region']=prod_final_adj['pop_gams']/prod_final_adj['pop_final_tot']


In [ ]:
#need to increase total skilled and unskilled spend by deflator

In [ ]:

###what percent is each pop of total demand in region
###contribute in proportion to relative percent demand of skill type in region
prod_final_adj['contribution_to_change']=prod_final_adj['raw_change']*prod_final_adj['share_labor_in_region']

prod_final_adj_2=prod_final_adj[['q','s','sk','pop_gams','expenditure_share','pop_final','deflator','raw_change','share_labor_in_region','contribution_to_change','pop_final_cons','prod_pop_actual']].sort_values(by=['q','s'])
prod_final_adj_2['pop_adjusted']=prod_final_adj_2['pop_final']+prod_final_adj_2['contribution_to_change']

prod_final_adj_2['skill_shr']=prod_final_adj_2['pop_adjusted']/prod_final_adj_2['pop_gams']



In [ ]:
prod_final_adj_2.head()

In [ ]:
cons_final.head()

In [ ]:
cons_final.groupby(['q','sk'],as_index=False).agg({'pop_final':sum}).head()

In [ ]:
prod_final_adj_2.groupby(['q','sk'],as_index=False).agg({'pop_adjusted':sum})

In [ ]:
####download actual ld0 to check
ld0_testpop=pd.read_csv('/Users/hannahkamen/Downloads/windc_ld0.csv')

ld0_testpop=ld0_testpop.rename(columns={' flx':'q',' int':'s',' scn':'pop'})
ld0_testpop=ld0_testpop[['q','s','pop']]


In [ ]:
prod_final_test=prod_final_adj[['','s','sk','skill_shr']]
prod_final_test=prod_final_test.rename(columns={'':'q'})

In [ ]:
prod_final_test=prod_final_test.merge(ld0_testpop,on=['q','s'])
prod_final_test['pop_test']=prod_final_test['skill_shr']*prod_final_test['pop']

In [ ]:
prod_final_test.groupby(['q','sk'],as_index=False).agg({'pop_test':sum}).head()

In [ ]:
rules_for_prod.head()

#### Export

In [ ]:
cons_final['']=cons_final['r']
cons_final['skill_shr']=cons_final['expenditure_share']
cons_final[['','q','h','sk','skill_shr']].to_csv('/Users/hannahkamen/Downloads/le0_shr2.csv',index=False)

In [ ]:
prod_final_adj_2['']=prod_final_adj['q']

prod_final_adj_2[['','s','sk','skill_shr']].to_csv('/Users/hannahkamen/Downloads/ld0_shr2.csv',index=False)

#### Get household specific deflator

In [ ]:
####get proportion household in their working region in GAMS data

le00_rh=le00.groupby(['census_region_q','h','q'],as_index=False).sum()
le00_rh=le00_rh.rename(columns={'pop':'pop_gams'})

le00_rh_tot=le00.groupby(['census_region_q','q'],as_index=False).sum()
le00_rh_tot=le00_rh_tot.rename(columns={'pop':'pop_total_gams'})
le00_rh=le00_rh.merge(le00_rh_tot,on=['census_region_q','q'])
le00_rh['prop_total_gams']=le00_rh['pop_gams']/le00_rh['pop_total_gams']

####get proportion household in their working region in ACS data

acs_cons=all_breakouts_df.groupby(['census_region_q','h','q'],as_index=False).sum()
del acs_cons['deflator']
acs_cons=acs_cons.rename(columns={'incwage':'pop'})
acs_cons_tot=acs_cons.groupby(['census_region_q','q'],as_index=False).sum()


acs_cons_tot=acs_cons_tot.rename(columns={'pop':'pop_total'})
acs_cons=acs_cons.merge(acs_cons_tot,on=['census_region_q','q'])
acs_cons['prop_total']=acs_cons['pop']/acs_cons['pop_total']

####get decompressor

cons_df=acs_cons.merge(le00_rh,on=['census_region_q','h','q'])
cons_df=cons_df[['census_region_q','h','q','']]

In [ ]:
acs_cons

#### Get expenditure shares by skill type household and sector
##### these breakouts will inform use, of the total endowment by household in each region and sector, what percentage is spent on skilled vs unskilled labor
#### Then get percent each household and sector is in GAMS data of total regional labor endowment 
##### so for example, will have something like HH1 has 10% of the total labor endowment for the midwest, of this labor endowment 60% is spent on skilled labor and 40% is spent on unskilled labor 

In [ ]:
####by household

all_breakouts_lm.groupby(['census_region_q','r','q','h'],as_index=False).agg({'incwage':sum})

####these breakouts 


In [ ]:
###get share of total labor in each census region by skill

tot_r=all_breakouts_lm.groupby(['census_region_q'],as_index=False).agg({'incwage':sum})
tot_r=tot_r.rename(columns={'incwage':'incwage_tot'})
tot_r_sk=all_breakouts_lm.groupby(['census_region_q','sk'],as_index=False).agg({'incwage':sum})

tot_r_sk=tot_r_sk.merge(tot_r,on=['census_region_q'],how='inner')
tot_r_sk['share_labor_region']=tot_r_sk['incwage']/tot_r_sk['incwage_tot']
del tot_r_sk['incwage']
del tot_r_sk['incwage_tot']

##get total labor endowment by region and skill in gams

le00_st=le00.groupby(['census_region_q'],as_index=False).sum()

tot_r_sk=tot_r_sk.merge(le00_st, on=['census_region_q'],how='right')

tot_r_sk['pop_skill_region']=tot_r_sk['share_labor_region']*tot_r_sk['pop']

##this dataset has the total labor endowment by skill in each region using the GAMS table populations by region:

tot_r_sk=tot_r_sk[['census_region_q','sk','pop_skill_region']]

##### get share of total skilled endowment in each region allocated to each household
##### this datset shows the percent of total skill type that each household is endowed with in each region

##### need to calibrate to actual householde population levels in gams data
1) get household populations in each region in gams data
2) multiply household populations by percent of skill type that each is endowed with
3) calculate breakouts this way

In [ ]:
#######get breakout of household by region in gams data

#get shares of total region population that each hopusehold comprises
le00_r=le00.groupby(['census_region_q'],as_index=False).sum()
le00_r=le00_r.rename(columns={'pop':'total_r_pop'})
le00_rh=le00.groupby(['census_region_q','h'],as_index=False).sum()
le00_rh=le00_rh.merge(le00_r,on='census_region_q')
le00_rh['regional_hh_share']=le00_rh['pop']/le00_rh['total_r_pop']
le00_rh=le00_rh[['census_region_q','h','pop']]

##the above dataset has the regional population by household as specified in GAMS data
### now get percent of total labor type endowed to each household in wage data
####get these values that do not reflect actual amounts in gams data
#####calculate shares from these values and apply to actual labor endowment by hh type in le00_rh


#######tot_r_sk_h_raw gives us the percentage of total labor endowment by skill type that each household
#########is endowed with in each region

tot_r_sk_raw=all_breakouts_lm.groupby(['census_region_q','sk'],as_index=False).agg({'incwage':sum})
tot_r_sk_raw=tot_r_sk_raw.rename(columns={'incwage':'incwage_tot'})
tot_r_sk_h_raw=all_breakouts_lm.groupby(['census_region_q','sk','h'],as_index=False).agg({'incwage':sum})

tot_r_sk_h_raw=tot_r_sk_h_raw.merge(tot_r_sk_raw,on=['census_region_q','sk'],how='inner')
tot_r_sk_h_raw['share_hh_bysk']=tot_r_sk_h_raw['incwage']/tot_r_sk_h_raw['incwage_tot']
del tot_r_sk_h_raw['incwage']
del tot_r_sk_h_raw['incwage_tot']

tot_r_sk_h_raw=tot_r_sk_h_raw[['sk','census_region_q','h','share_hh_bysk']]
#....HH1 GETS 1% OF SKILLED LABOR IN MIDWEST AND SO ON.....
#NOW NEED SHARE OF TOTAL LABOR THAT SKILLED AND UNSKILLED IS IN EACH REGION. SET THIS RATIO IN GAMS DATA
# tot_r_sk TELLS US TOTAL LABOR IN EACH REGION by EACH SKILL TYPE AND SUMS TO TOTAL LABOR IN GAMS

#SO USING tot_r_sk_h_raw WE CAN SAY HH1 GETS 1% OF SKILLED LABOR IN REGION WHICH IS 1% of 61.91% OF THE TOTAL POP

#ASSIGN THESE LVEL VALUES TO HH1, THEN RECALCULATE SHARES


###HH1 is 10% of total labor demand in region in gams, 
### HH1 demands 1% total unskilled and 2% of total skilled labor in the region and 10% of total labor
###this woould put them at 50 demanded but gams says they only get 28 units endowed
### HH1 can only be 28 percent of total demand in gams

#...okay so need to calibrate to industry size in GAMS what is 10% of total labor in region, 28 units 

####merge on total labor endowment by region 

tot_r_sk_lm=tot_r_sk[['census_region_q','sk','pop_skill_region']]

get_hh_splits=tot_r_sk_h_raw.merge(tot_r_sk_lm,on=['census_region_q','sk'])

###multiply the percent of total labor in region by skill each household gets by total endow of labor type in that region
get_hh_splits['hh_endow_bysk']=get_hh_splits['share_hh_bysk']*get_hh_splits['pop_skill_region']

tot_hh_endow=get_hh_splits.groupby(['census_region_q','h'],as_index=False).agg({'hh_endow_bysk':sum})

tot_hh_endow=tot_hh_endow.rename(columns={'hh_endow_bysk':'tot_hh_endowment'})
tot_hh_endow=tot_hh_endow[['census_region_q','h','tot_hh_endowment']]
get_hh_splits=get_hh_splits.merge(tot_hh_endow,on=['census_region_q','h'])

#### get expenditure share by skill type 
get_hh_splits['expenditure_share']=get_hh_splits['hh_endow_bysk']/get_hh_splits['tot_hh_endowment']
get_hh_splits=get_hh_splits[['sk','census_region_q','h','expenditure_share']]
####get actual household labor endowment from gams

cons_final=get_hh_splits.merge(le00_rh,on=['census_region_q','h'])
cons_final['pop_hh_sk']=cons_final['expenditure_share']*cons_final['pop']
# ####merge share skill type by household onto previous table expressing share of labor type in each region

# cons_final=tot_r_sk_h_raw.merge(tot_r_sk,on=['census_region_q','sk'])
# cons_final['pop_region_skill_hh']=cons_final['share_hh_bysk']*cons_final['pop_skill_region']
# cons_final=cons_final[['census_region_q','h','sk','pop_region_skill_hh','share_hh_bysk']]


In [ ]:
tot_r_sk_cal=cons_final.groupby(['sk','census_region_q'],as_index=False).agg({'pop_hh_sk':sum})
tot_r_sk_cal=tot_r_sk_cal.rename(columns={'pop_hh_sk':'pop_skill_region'})

In [ ]:
tot_r_sk_cal


##### get share of total skilled endowment in each region allocated to each sector

In [ ]:
#######get breakout of sector by region in gams data

#get shares of total region population that each hopusehold comprises
ld00_r=ld00.groupby(['census_region_q'],as_index=False).sum()
ld00_r=ld00_r.rename(columns={'pop':'total_r_pop'})
ld00_rh=ld00.groupby(['census_region_q','s'],as_index=False).sum()
ld00_rh=ld00_rh.merge(ld00_r,on='census_region_q')
ld00_rh['regional_s_share']=ld00_rh['pop']/ld00_rh['total_r_pop']
ld00_rh=ld00_rh[['census_region_q','s','pop']]

##the above dataset has the regional demand by sector as specified in GAMS data
### now get percent of total demand by sector in wage data by skill type
####get these values of demand that do not reflect actual amounts in gams data
#####calculate shares from these values and apply to actual labor endowment by s type in ld00_rh


#######tot_r_sk_s_raw gives us the percentage of total sector demand by skill typein each region

tot_r_sk_raw=all_breakouts_lm.groupby(['census_region_q','sk'],as_index=False).agg({'incwage':sum})
tot_r_sk_raw=tot_r_sk_raw.rename(columns={'incwage':'incwage_tot'})
tot_r_sk_s_raw=all_breakouts_lm.groupby(['census_region_q','sk','s'],as_index=False).agg({'incwage':sum})

tot_r_sk_s_raw=tot_r_sk_s_raw.merge(tot_r_sk_raw,on=['census_region_q','sk'],how='inner')
tot_r_sk_s_raw['share_s_bysk']=tot_r_sk_s_raw['incwage']/tot_r_sk_s_raw['incwage_tot']
del tot_r_sk_s_raw['incwage']
del tot_r_sk_s_raw['incwage_tot']

tot_r_sk_s_raw=tot_r_sk_s_raw[['sk','census_region_q','s','share_s_bysk']]

####merge on total labor endowment by region and skill type

tot_r_sk_lm=tot_r_sk[['census_region_q','sk','pop_skill_region']]

get_s_splits=tot_r_sk_s_raw.merge(tot_r_sk_lm,on=['census_region_q','sk'])

###multiply the percent of total labor in region by skill each household gets by total endow of labor type in that region
get_s_splits['s_endow_bysk']=get_s_splits['share_s_bysk']*get_s_splits['pop_skill_region']

tot_s_endow=get_s_splits.groupby(['census_region_q','s'],as_index=False).agg({'s_endow_bysk':sum})

tot_s_endow=tot_s_endow.rename(columns={'s_endow_bysk':'tot_s_endowment'})
tot_s_endow=tot_s_endow[['census_region_q','s','tot_s_endowment']]
get_s_splits=get_s_splits.merge(tot_s_endow,on=['census_region_q','s'])

#### get expenditure share by skill type 
get_s_splits['expenditure_share']=get_s_splits['s_endow_bysk']/get_s_splits['tot_s_endowment']
get_s_splits=get_s_splits[['sk','census_region_q','s','expenditure_share']]
####get actual household labor endowment from gams

prod_final=get_s_splits.merge(ld00_rh,on=['census_region_q','s'])
prod_final['pop_s_sk']=prod_final['expenditure_share']*prod_final['pop']



In [ ]:
tot_r_sk_s_raw

In [ ]:
tot_r_sk_cal=prod_final.groupby(['sk','census_region_q'],as_index=False).agg({'pop_s_sk':sum})

In [ ]:
tot_r_sk_cal

In [ ]:
####ok these datasets match on total pop, total pop by sector and total pop by household as defined in gams
####these datasets do not clear on skilled and unskilled labor demanded

##for consumer, we have the population by household for each census region
### 1) calculate percentage of total labor type in region demanded by household
### 2) calculate percentage of total labor type in region demanded by household
### 3) turn these into raw values based on gams percentages
### 4) calculate shares


In [ ]:
tot_r_sk_raw

In [ ]:
prod_final.groupby(['census_region_q','s'],as_index=False).sum()

In [ ]:
ld00.groupby(['census_region_q','s'],as_index=False).sum()

In [ ]:
prod_final.head()

In [ ]:
# #######get breakout of sector by skill type in wage data

# tot_r_sk_raw=all_breakouts_lm.groupby(['census_region_q','sk'],as_index=False).agg({'incwage':sum})
# tot_r_sk_raw=tot_r_sk_raw.rename(columns={'incwage':'incwage_tot'})
# tot_r_sk_h_raw=all_breakouts_lm.groupby(['census_region_q','sk','s'],as_index=False).agg({'incwage':sum})

# tot_r_sk_h_raw=tot_r_sk_h_raw.merge(tot_r_sk_raw,on=['census_region_q','sk'],how='inner')
# tot_r_sk_h_raw['share_s_bysk']=tot_r_sk_h_raw['incwage']/tot_r_sk_h_raw['incwage_tot']
# del tot_r_sk_h_raw['incwage']
# del tot_r_sk_h_raw['incwage_tot']

# ####merge share skill type by household onto previous table expressing share of labor type in each region

# prod_final=tot_r_sk_h_raw.merge(tot_r_sk,on=['census_region_q','sk'])
# prod_final['pop_region_skill_s']=prod_final['share_s_bysk']*prod_final['pop_skill_region']
# prod_final=prod_final[['census_region_q','s','sk','pop_region_skill_s','share_s_bysk']]

In [ ]:
##### now have:

1) endowment of each household by skill type
2) demand of each sector by skill type 
##### in this section I re-scale to create shares that are realtive to the total labor endowment by each skill type. I sum total demand by household and sector based on allocations based on percentage of total labor type, then create shares by these sums for each household and sector:
i.e. sector "adm" demands 1.11% of total skilled labor  and 2.2% of total unskilled labor. This sums to demands of 12.172 and 14.892 respectivly. Baase on this, their expendditure breakout is 12.172/12.172 + 14.892 and 14.892/12.172 + 14.892 for skilled and unskilled respectively.




##### now have:

1) datasets showing the percentage of skilled and unskilled labor totals in each region that each household is endowed with. i.e. in the midwest, hh1 makes up 3% of skilled labor,  and 13.5% of unskilled labor

2) datasets showing the percentage of skilled and unskilled labor totals in each region that each sector is demands.
   i.e. in the midwest, sector "adm" demands 1% of skilled labor and 2.2% of unskilled labor.
   
##### in this section I re-scale to create shares that are realtive to the total labor endowment by each skill type. I sum total demand by household and sector based on allocations based on percentage of total labor type, then create shares by these sums for each household and sector:
i.e. sector "adm" demands 1.11% of total skilled labor  and 2.2% of total unskilled labor. This sums to demands of 12.172 and 14.892 respectivly. Baase on this, their expendditure breakout is 12.172/12.172 + 14.892 and 14.892/12.172 + 14.892 for skilled and unskilled respectively.

#### Consumer side

In [ ]:
####get total endowment by household
cons_sumbyhh=cons_final.groupby(['census_region_q','h'],as_index=False).agg({'pop_region_skill_hh':sum})
cons_sumbyhh=cons_sumbyhh.rename(columns={'pop_region_skill_hh':'pop_region_hh'})

###merge on to pop splits by skill
cons_final=cons_final.merge(cons_sumbyhh,on=['census_region_q','h'])

###recalculate expenditure shares based on percentage of total endowment allocated
cons_final['expenditure_shares']=cons_final['pop_region_skill_hh']/cons_final['pop_region_hh']
cons_final_lm=cons_final[['census_region_q','h','sk','pop_region_skill_hh','expenditure_shares']]

In [ ]:
####get total endowment by sector
prod_sumbys=prod_final.groupby(['census_region_q','s'],as_index=False).agg({'pop_region_skill_s':sum})
prod_sumbys=prod_sumbys.rename(columns={'pop_region_skill_s':'pop_region_s'})

###merge on to pop splits by skill
prod_final=prod_final.merge(prod_sumbys,on=['census_region_q','s'])

###recalculate expenditure shares based on percentage of total demand allocated
prod_final['expenditure_shares']=prod_final['pop_region_skill_s']/prod_final['pop_region_s']
prod_final_lm=prod_final[['census_region_q','s','sk','pop_region_skill_s','expenditure_shares']]

In [ ]:
cons_final_lm.groupby(['census_region_q','h'],as_index=False).agg({'pop_region_skill_hh':sum})['pop_region_skill_hh'].sum()




In [ ]:
le00.groupby(['h','census_region_r'],as_index=False).sum()['pop'].sum()

In [ ]:
ld00.groupby(['sk','census_region_q'],as_index=False).sum().head()

##### now merge on all combinations for consumer and producer side

In [ ]:
####consumers
####group labor endowment dataset across skill to get "pop" by r,q,h

le00_sum=le00.groupby(['r','q','h','census_region_q'],as_index=False).sum()

####merge this dataset with consumer endowment shares by skill

cons_breakouts=le00_sum.merge(cons_final_lm,on=['census_region_q','h'])

cons_breakouts['final_pop']=cons_breakouts['pop']*cons_breakouts['expenditure_shares']
cons_breakouts_final=cons_breakouts[['r','q','sk','h','census_region_q','expenditure_shares','final_pop']]


In [ ]:
####consumers
####group labor endowment dataset across skill to get "pop" by r,q,h

ld00_sum=ld00.groupby(['q','s','census_region_q'],as_index=False).sum()

####merge this dataset with producer expenditure shares by skill

prod_breakouts=ld00_sum.merge(prod_final_lm,on=['census_region_q','s'])

prod_breakouts['final_pop']=prod_breakouts['pop']*prod_breakouts['expenditure_shares']
prod_breakouts_final=prod_breakouts[['q','s','sk','census_region_q','expenditure_shares','final_pop']]


In [ ]:
cons_breakouts_final.groupby(['census_region_q','sk'],as_index=False).agg({'final_pop':sum})

In [ ]:
prod_breakouts_final.groupby(['census_region_q','sk'],as_index=False).agg({'final_pop':sum})

In [ ]:
#share of total regional skilled labor each sector demands

In [ ]:
#share of total regional skilled labor each household endows
#share of household endowment by labor type

In [ ]:
household 1 makes up 10% of total labor endowment, of that 10%, 40% is skilled, 60% is unskilled

In [ ]:
sector j makes up 10% of total labor demand, of that 10%, 40% is skilled, 60% is unskilled

In [ ]:
prod_final.head()

In [ ]:
le00_s['pop'].sum()

In [ ]:
ca_rqhsk['pop'].sum()

In [ ]:
#### merge on final dataset prod

ld00_s=ld00.groupby(['q','s','census_region_q'],as_index=False).sum()

all_breakouts_aug_prod=all_breakouts_lm.merge(ld00, on=['q','s','sk','census_region_q'],how='right')

del all_breakouts_aug_prod['pop']

all_breakouts_aug_prod2=all_breakouts_aug_prod.merge(ld00_s, on=['q','s','census_region_q'],how='outer',indicator=True)



# merge census averages onto original merged dataframe
ca_qssk=all_breakouts_aug_prod2.merge(prod_final,on=['census_region_q','s','sk'],how='inner')
ca_qssk=ca_qssk.groupby(['q','s','sk','census_region_q'],as_index=False).agg({'census_share':max,'pop':max})
ca_qssk['pop_final']=ca_qssk['census_share']*ca_qssk['pop']


In [ ]:
ca_qssk.groupby(['sk','census_region_q'],as_index=False).agg({'pop_final':sum})

In [ ]:
ca_rqhsk.groupby(['sk','census_region_q'],as_index=False).agg({'pop_final':sum})

In [ ]:
#get percent of all labor in region that a household comprises
#i.e. midwest hh1 is 10%
#get percent breakout that each household is of skilled and unskilled
# 

In [ ]:
#of skilled labor in the midwest, hh1 is 10%
# hh1 units are 40% skilled 60% unskilled

In [ ]:
prod_final

In [ ]:
#of the skilled expenditures demanded in the west, adm gets 0.01 pct
#of the unskilled labor expenditures in the west, adm gets 0.004 pct

#adm gets a total of (0.01*1789.7872029262608)+(0.004*1789.7872029262608) = 25.057 units

#but data says he gets 51

#rescale total expenditures in west to be this large .014x=51, =3642.8571428571427



In [ ]:
#in gams table, allocate 1% of total skilled expenditures to adm skilled
#in gams table, allocate 4% of total unskilled expenditures to adm unskilled

#get that total, get that breakout

# recalculate breakout on actual labor endowment

In [ ]:
51/.014

In [ ]:
ld00[(ld00['census_region_q']=='west')& (ld00['sk']=='skl')]

In [ ]:
(0.01*1789.7872029262608)+(0.004*1789.7872029262608)

In [ ]:
prod_final

In [ ]:
ca_qssk[(ca_qssk['census_region_q']=='west')& (ca_qssk['sk']=='skl')]

In [ ]:
cons_final.head()

In [ ]:
prod_final.groupby(['census_region_q','sk'],as_index=False).sum()

In [ ]:
prod_final

In [ ]:
cons_final.groupby(['census_region_q','sk'],as_index=False).sum()

#### Assign census shares to all breakouts

In [ ]:
### for r,q,h,sk breakouts get skill shares of expenditures by census region working in

census_avg_tot0=all_breakouts_lm.groupby(['census_region_q','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=all_breakouts_lm.groupby(['census_region_q','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region_q','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

le00_s=le00.groupby(['r','q','h','census_region_q'],as_index=False).sum()

all_breakouts_aug_cons=all_breakouts_lm.merge(le00, on=['r','q','h','sk','census_region_q'],how='right')
del all_breakouts_aug_cons['pop']
all_breakouts_aug_cons2=all_breakouts_aug_cons.merge(le00_s, on=['r','q','h','census_region_q'],how='outer',indicator=True)


# merge census averages onto original merged dataframe
ca_rqhsk=all_breakouts_aug_cons2.merge(census_avg0,on=['census_region_q','h','sk'],how='inner')
ca_rqhsk=ca_rqhsk.groupby(['r','q','h','sk','census_region_q'],as_index=False).agg({'census_share':max,'pop':max})
ca_rqhsk['new_pop']=ca_rqhsk['census_share']*ca_rqhsk['pop']


###########################
###########################


### for r,s,sk breakouts get skill shares of expenditures by census region industry is in 

census_avg_tot1=all_breakouts_lm.groupby(['census_region_q','s'],as_index=False).agg({'incwage':sum})
census_avg_tot1=census_avg_tot1.rename(columns={'incwage':'incwage_tot'})
census_avg1=all_breakouts_lm.groupby(['census_region_q','s','sk'],as_index=False).agg({'incwage':sum})


census_avg1=census_avg1.merge(census_avg_tot1,on=['census_region_q','s'],how='inner')
census_avg1['census_share']=census_avg1['incwage']/census_avg1['incwage_tot']
del census_avg1['incwage']
del census_avg1['incwage_tot']

ld00_s=ld00.groupby(['q','s','census_region_q'],as_index=False).sum()

all_breakouts_aug_prod=all_breakouts_lm.merge(ld00, on=['q','s','sk','census_region_q'],how='right')
del all_breakouts_aug_prod['pop']
all_breakouts_aug_prod2=all_breakouts_aug_prod.merge(ld00_s, on=['q','s','census_region_q'],how='outer',indicator=True)



# merge census averages onto original merged dataframe
ca_qssk=all_breakouts_aug_prod2.merge(census_avg1,on=['census_region_q','s','sk'],how='inner')
ca_qssk=ca_qssk.groupby(['q','s','sk','census_region_q'],as_index=False).agg({'census_share':max,'pop':max})
ca_qssk['new_pop']=ca_qssk['census_share']*ca_qssk['pop']


In [ ]:
ca_qssk['new_pop'].sum()

In [ ]:
ca_rqhsk['new_pop'].sum()

##### ak, hi, co, wa, ca, oil sector all demand census level share of oil demand for unskilled labor

##### ak, hi, co, wa, ca, hh1 all have census level share of hh1 unskilled labor 

In [ ]:
tot=ca_rqhsk[(ca_rqhsk['q'].isin(west)) & (ca_rqhsk['h'].isin(['hh1']))]['new_pop'].sum()

unskl=ca_rqhsk[(ca_rqhsk['q'].isin(west)) & (ca_rqhsk['h'].isin(['hh1'])) & (ca_rqhsk['sk']=='unskl')]['new_pop'].sum()

In [ ]:
ca_rqhsk[(ca_rqhsk['q'].isin(west)) & (ca_rqhsk['h'].isin(['hh1']))]

In [ ]:
ca_rqhsk[(ca_rqhsk['q'].isin(south))&(ca_rqhsk['r'].isin(south))  ].to_excel('/Users/hannahkamen/Downloads/h_test.xlsx')


In [ ]:
ca_qssk[(ca_qssk['q'].isin(south))  ].to_excel('/Users/hannahkamen/Downloads/s_test.xlsx')




In [ ]:
ca_rqhsk[(ca_rqhsk['q'].isin(west)) &  (ca_rqhsk['sk'].isin(['skl']))]['new_pop'].sum()

In [ ]:
tot=ca_qssk[(ca_qssk['q'].isin(west)) & (ca_qssk['s'].isin(['adm']))]['new_pop'].sum()

unskl=ca_qssk[(ca_qssk['q'].isin(west)) & (ca_qssk['s'].isin(['adm'])) & (ca_qssk['sk']=='unskl')]['new_pop'].sum()


ca_qssk[(ca_qssk['q'].isin(west)) & (ca_qssk['s'].isin(['adm']))]

In [ ]:
0.449746+0.550254

In [ ]:
ca_qssk.groupby(['s','sk','census_region_q'],as_index=False).agg({'census_share':max})

In [ ]:
unskl/tot

In [ ]:
ca_rqhsk.groupby(['q','census_region_q'],as_index=False).sum()[['q','new_pop']]

In [ ]:
ca_qssk.groupby(['q','census_region_q'],as_index=False).sum()[['q','new_pop','census_region_q']]

In [ ]:
each of r,q,h

In [ ]:
##outer merge CPS data with gams table with indicator
merge_gams=gams_rqhs_lm1.merge(le00,on=['r','q','h','sk'],how='outer',indicator=True)
###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_gams['census_region']=np.where(merge_gams['r'].isin(midwest),'midwest','')
merge_gams['census_region']=np.where(merge_gams['r'].isin(south),'south',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(west),'west',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(northeast),'northeast',merge_gams['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot0=merge_gams.groupby(['census_region','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=merge_gams.groupby(['census_region','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

# merge census averages onto original merged dataframe
merge_gams_cs=merge_gams.merge(census_avg0,on=['census_region','h','sk'],how='inner')

##keep only inner values and values missing from CPS

final0=merge_gams_cs[merge_gams_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot0=final0.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
final_tot0=final_tot0.rename(columns={'incwage':'incwage_tot'})


final_shrs0=final0.merge(final_tot0,on=['r','q','h'],how='inner')
final_shrs0['skill_shr']=final_shrs0['incwage']/final_shrs0['incwage_tot']
del final_shrs0['incwage']
del final_shrs0['incwage_tot']

#create tag to define missing combinations in final dataset

final_shrs0['combo']=final_shrs0['r']+"_"+final_shrs0['q']+"_"+final_shrs0['h']+"_"+final_shrs0['sk']

## get list of missing combionations using _merge indicator var

missing_combos0=final_shrs0[final_shrs0['_merge']=='right_only']['r']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['q']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['h']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['sk']

# ###if region sector combination is not missing, use census region values, otherwise keep original share value
# final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(missing_combos0),final_shrs0['census_share'],final_shrs0['skill_shr'])
# final_shrs0['skill_shr']=np.where(final_shrs0['combo']=='ME_NJ_hh1',final_shrs0['census_share'],final_shrs0['skill_shr'])




# ###limit dataframe

# final_shrs0=final_shrs0[['r','q','s','h','sk','skill_shr','combo','pop','census_share']]

# #chksum to see that all proportions add to 1
# check_sum0=final_shrs0.groupby(['r','q','s','h'],as_index=False).agg({'skill_shr':sum})
# check_sum0=check_sum0[check_sum0['skill_shr']!=1]


# #chksum to see that no zeros exist
# check_sum10=final_shrs0.groupby(['r','q','s','h','sk'],as_index=False).agg({'skill_shr':sum})
# check_sum10=check_sum10[check_sum10['skill_shr']==0]

# zeros=check_sum10['r']+"_"+check_sum10['q']+"_"+check_sum10['s']+"_"+check_sum10['h']


# #replace 0s with census values

# final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(zeros),final_shrs0['census_share'],final_shrs0['skill_shr'])



# #check zeros agains
# check_sum10=final_shrs0.groupby(['r','q','s','h','sk'],as_index=False).agg({'skill_shr':sum})
# check_sum10=check_sum10[check_sum10['skill_shr']==0]

# final_shrs0_f=final_shrs0[['r','q','s','h','sk','skill_shr']]



In [ ]:
missing_combos0

In [ ]:
final_shrs0[final_shrs0['census_share']==0]

In [ ]:
zeros

In [ ]:
check_sum10

In [ ]:
missing_combos0

In [ ]:
check_sum0

In [ ]:
final_shrs0[final_shrs0['combo'].isin(missing_combos0)]['pop'].sum()

In [ ]:
469.6733563736173/9215

In [ ]:
len(final_shrs0)

In [ ]:
missing_combos0